In [1]:
# %pip install python-dotenv


In [2]:
import os
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import tensorflow as tf
import sqlalchemy as sq
import sklearn as sk
from dotenv import load_dotenv
from IPython.display import clear_output
from matplotlib import pyplot as plt
from DataService import DataService

2022-11-27 04:52:12.338982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-27 04:52:12.457801: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# The following lines adjust the granularity of reporting.
#pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir('/tf')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')
NULLFLAG = -9999

In [4]:
# Connect to the database
pullService = DataService(PGDB, PGUSER, PGPW)
db_pull_con = pullService.connect()

pushService = DataService(PGDB, PGUSER, PGPW)
db_push_con = pushService.connect()

In [5]:
# Load the data
fireWeather = "FireWeather"
query = "SELECT * FROM public.\"{}\";".format(fireWeather)
dfEval = pd.read_sql(query, db_pull_con)

In [6]:
dfEval.count()

EntryID                 428
FIRE_ID                 428
FIRENAME                428
YEAR                    428
MONTH                   428
DAY                     428
REP_DATE                428
SIZE_HA                 428
size_ha_bin             428
ClimateID               428
OneYear                 428
OneMonth                428
OneMeanTemp             428
OneMinTemp              428
OneMaxTemp              428
OneMeanDewPoint         428
OneMinDewPoint          428
OneMaxDewPoint          428
OneMeanHumidity         428
OneMinHumidity          428
OneMaxHumidity          428
OneMeanPressure         426
OneMinPressure          426
OneMaxPressure          426
OneMeanWindSpeed        426
OneMinWindSpeed         426
OneMaxWindSpeed         426
OneMeanWindChill        426
OneMinWindChill         426
OneMaxWindChill         426
OneTotalPrecip          428
OneMeanWindDirection    426
TwoYear                 428
TwoMonth                428
TwoMeanTemp             428
TwoMinTemp          

In [7]:
dfEval.describe()

,EntryID,YEAR,MONTH,DAY,SIZE_HA,size_ha_bin,OneYear,OneMonth,OneMeanTemp,OneMinTemp,OneMaxTemp,OneMeanDewPoint,OneMinDewPoint,OneMaxDewPoint,OneMeanHumidity,OneMinHumidity,OneMaxHumidity,OneMeanPressure,OneMinPressure,OneMaxPressure,OneMeanWindSpeed,OneMinWindSpeed,OneMaxWindSpeed,OneMeanWindChill,OneMinWindChill,OneMaxWindChill,OneTotalPrecip,OneMeanWindDirection,TwoYear,TwoMonth,TwoMeanTemp,TwoMinTemp,TwoMaxTemp,TwoMeanDewPoint,TwoMinDewPoint,TwoMaxDewPoint,TwoMeanHumidity,TwoMinHumidity,TwoMaxHumidity,TwoMeanPressure,TwoMinPressure,TwoMaxPressure,TwoMeanWindSpeed,TwoMinWindSpeed,TwoMaxWindSpeed,TwoMeanWindChill,TwoMinWindChill,TwoMaxWindChill,TwoTotalPrecip,TwoMeanWindDirection
count,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,426.0,426.0,426.0,426.0,426.0,426.0,426.0,426.0,426.0,428.0,426.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,428.0,426.0,426.0,426.0,426.0,426.0,426.0,426.0,426.0,426.0,428.0,426.0
mean,3829.2,2013.9,6.8,15.9,8936.9,1.7,2013.9,5.8,12.5,-2.8,27.3,3.6,-7.5,13.3,60.0,18.5,97.2,97.4,95.8,98.7,9.2,0.0,29.5,-18.1,-19.1,-7.9,22.9,16.0,2013.9,4.8,5.3,-12.1,22.6,-3.0,-16.8,8.0,60.0,18.6,96.8,97.5,95.6,98.8,9.7,0.1,30.8,-16.7,-23.1,-3.3,16.8,15.3
std,3843.2,2.6,0.9,9.8,32731.6,1.1,2.6,0.9,4.7,6.6,3.2,5.4,6.9,4.4,7.1,6.2,2.6,0.9,1.0,1.0,1.9,0.2,6.3,2.3,2.7,8.9,29.1,2.9,2.6,0.9,6.9,11.1,6.0,6.2,10.4,5.1,6.5,6.9,3.7,1.0,1.0,1.1,2.0,0.6,6.8,2.4,7.3,6.1,21.5,2.4
min,33.0,2010.0,5.0,1.0,200.7,0.0,2010.0,4.0,-6.3,-28.4,6.2,-12.4,-31.9,-4.6,38.0,10.0,81.1,95.1,93.2,95.9,4.3,0.0,16.0,-24.4,-35.0,-24.4,0.0,9.2,2010.0,3.0,-16.1,-39.5,-0.3,-21.3,-43.8,-10.5,43.6,10.0,71.9,95.2,92.4,96.4,5.5,0.0,19.0,-23.8,-47.0,-22.3,0.0,9.2
25%,757.5,2012.0,6.0,6.0,651.8,1.0,2012.0,5.0,8.7,-7.6,26.0,-1.5,-11.7,10.4,54.7,15.0,96.0,96.8,95.3,98.0,7.8,0.0,24.0,-19.9,-20.4,-18.1,0.0,14.0,2012.0,4.0,0.3,-21.0,19.9,-7.5,-24.0,4.3,55.3,15.0,96.0,96.9,94.9,98.0,8.1,0.0,26.0,-18.8,-27.0,-2.0,0.0,13.6
50%,1736.0,2014.0,7.0,15.0,1880.7,2.0,2014.0,6.0,14.0,-1.6,27.5,5.2,-5.2,14.7,60.3,18.0,97.0,97.4,95.8,98.7,9.2,0.0,30.0,-17.9,-18.4,-1.0,9.9,15.6,2014.0,5.0,6.9,-8.5,24.1,-2.1,-14.9,8.5,60.2,17.0,97.0,97.5,95.7,98.8,9.6,0.0,30.0,-16.4,-20.0,-1.0,8.3,15.4
75%,8700.2,2016.0,7.0,25.2,5092.8,3.0,2016.0,6.0,15.9,1.6,29.4,8.2,-2.3,16.7,65.4,22.0,99.0,97.9,96.4,99.4,10.5,0.0,33.0,-16.5,-17.3,-1.0,40.1,17.6,2016.0,5.0,10.2,-3.6,26.9,0.4,-9.7,12.0,64.5,21.0,99.0,97.9,96.3,99.5,10.8,0.0,34.0,-15.1,-18.0,-1.0,29.3,16.9
max,12248.0,2020.0,10.0,31.0,491429.4,3.0,2020.0,9.0,19.2,10.1,33.4,12.3,5.0,20.5,81.1,81.1,100.0,99.4,97.8,102.0,14.0,2.0,49.0,-12.4,-14.9,0.0,175.9,22.9,2020.0,8.0,18.2,7.1,33.3,11.4,2.8,19.1,81.1,81.1,100.0,99.5,97.7,101.4,18.1,6.0,51.0,-10.7,-15.3,0.0,107.1,22.9
